This notebook benchmarks the memory-efficiency, inference speed, and accuracy in downstream tasks of Llama 2 7B, 13B, and Mistral 7B, all quantized in 2-bit, 3-bit, 4-bit, and 8-bit with GPTQ.



We will need the following libraries to benchmark with optimum-benchmark:

In [ ]:
!pip install --upgrade transformers auto-gptq accelerate datasets bitsandbytes
!python -m pip install git+https://github.com/huggingface/optimum.git
!python -m pip install git+https://github.com/huggingface/optimum-benchmark.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Clo

I use optimum-benchmark to track the memory consumption. Note that it also produces data about the inference speed.

In the following example of configuration file, I use Llama 13B. Replace "Llama-2-13b-hf" by "Llama-2-7b-hf" or by "Mistral-7B-v0.1" to benchmark the other models.

Results will be stored in experiments_ob/

In [ ]:
import os
for w in [2,3,4,8]:
  YAML_DEFAULT="""
  defaults:
    - backend: pytorch # default backend
    - benchmark: inference # we will monitor the inference
    - launcher: process
    - experiment # inheriting from experiment config
    - _self_ # for hydra 1.1 compatibility
    - override hydra/job_logging: colorlog # colorful logging
    - override hydra/hydra_logging: colorlog # colorful logging

  hydra:
    run:
      dir: experiments_ob/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
    sweep:
      dir: experiments_ob/${experiment_name}
    job:
      chdir: true
      env_set: #These are environment variable that you may want to set before running the benchmark
        CUDA_VISIBLE_DEVICES: 0
        CUDA_DEVICE_ORDER: PCI_BUS_ID

  experiment_name: kaitchup/Llama-2-13b-hf-gptq-%sbit
  model: kaitchup/Llama-2-13b-hf-gptq-%sbit #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
  device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

  backend:
    torch_dtype: float16

  benchmark:
    memory: true #We will monitor the memory usage
    warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

    new_tokens: 1000 #Inference will generate 1000 tokens
    input_shapes:
      sequence_length: 512 #Prompt will have 512 tokens
      batch_size: 2
  """ % (str(w), str(w))

  with open("llama2_13b_ob.yaml", 'w') as f:
    f.write(YAML_DEFAULT)
  os.system("optimum-benchmark --config-dir ./ --config-name llama2_13b_ob")

We will need the following libraries to benchmark with the Evaluation Harness:

In [ ]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git
!pip install bitsandbytes
!pip install --upgrade transformers
!pip install auto-gptq optimum

  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git to /tmp/pip-req-build-0_4_vubf
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-0_4_vubf
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit 45a8f709cd3fa903a0e2ff7275694d441bcf0cac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 17.4 MB/s eta 0:00:00
  Preparing meta

Here is an example of benchmarking with Winogrande, HellaSwag, and Arc Challenge tasks using 5-shot prompting. The results are stored in eval_harness/

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/Mistral-7B-v0.1-gptq-4bit --tasks winogrande,hellaswag,arc_challenge --device cuda:0 --num_fewshot 5 --batch_size 2 --output_path ./eval_harness/Mistral-7B-v0.1-gptq-4bit

2024-01-23:13:55:37,530 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-23:13:55:37,791 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-01-23:13:55:37,792 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-01-23:13:55:37,793 INFO     [config.py:108] JAX version 0.4.23 available.
2024-01-23 13:55:38.458320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 13:55:38.458379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 13:55:38.459763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 13:55:39.567973: W ten

# Appendix: Quantize with GPTQ

I used the following code to quantize Llama 2 7B, 13B, and Mistral-7B-v0.1 in 8,4, 3, and 2=bit.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer
import torch
model_path = 'mistralai/Mistral-7B-v0.1'

for w in [2,3,4,8]:
  quant_path = 'Mistral-7B-v0.1-gptq-'+str(w)+'bit'

  # Load model and tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
  model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
  quantizer = GPTQQuantizer(bits=w, dataset="c4", model_seqlen = 4096)
  quantized_model = quantizer.quantize_model(model, tokenizer)
  # Save quantized model and push the model to the HF Hub
  quantized_model.push_to_hub("kaitchup/"+quant_path)
  tokenizer.push_to_hub("kaitchup/"+quant_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00016-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]